<img src=banner.png>

# <a name="0">Measuring and Mitigating Toxicity in Large Language Models</a>

Building and operating machine learning applications responsibly requires an active, consistent approach to prevent, assess, and mitigate harm. This workshop guides you through how to identify toxicity in LLM generated summaries and how to mitigate and reduce toxicity.

In this workshop you will:
1. <a href="#1">Load a dataset</a>
2. <a href="#2">Load and use a Large Language Model (LLM)</a>
3. <a href="#3">Evaluate LLM generated summaries for toxicity</a>
4. <a href="#4">Reduce toxicity using a Direct Optimization Policy (DPO)</a>
5. <a href="#5">Evaluate</a>


**Learning Objectives**

In this workshop you will learn to:

- Measure and understand toxicity
- Apply toxicity metrics
- Compare results across evaluation datasets
- Mitigate toxicity with a direct optimization approach

**Runtime**

This notebook takes about 90 minutes to complete (using some inbuilt shortcuts).

### Let's get started

Start by upgrading [pip](https://pypi.org/project/pip/) (a Python package management system) and install all required libraries from the provided requirements.txt file.

In [2]:
# !pip install -q -U pip --root-user-action=ignore
# !pip3 install -q -r requirements.txt --root-user-action=ignore

In [3]:
import warnings
warnings.filterwarnings(
        action='ignore',
        category=UserWarning,
    )

import transformers, torch
transformers.logging.set_verbosity_error()

from tqdm.auto import tqdm as notebook_tqdm

# <a name="1">1. Load a dataset</a>
(<a href="#0">Go to top</a>)

In this notebook, you will be working with the "[Cornell Movie-Dialogs Corpus](https://convokit.cornell.edu/documentation/movie.html)", a large metadata-rich collection of fictional conversations extracted from raw movie scripts. The dataset contains 220,579 conversational exchanges between 10,292 pairs of movie characters in 617 movies.


In [4]:
from utils.data_utils import _prepare_data

# load the data
movie_df = _prepare_data()

# show the data
movie_df.head(2)

,movie,dialogue,genre
0,"""murderland""","Jesus, my legs are asleep. I'll never be able ...",crime
1,10 things i hate about you,They do not! They do to! I hope so. She okay? ...,comedy


**LLMs require the data to be stored in a specific format**; use the [HuggingFace 🤗 Datasets](https://huggingface.co/docs/datasets/index) library to convert the dataframe.

In [5]:
from datasets import Dataset

# convert the data
movie_dataset = Dataset.from_pandas(movie_df)

# show the data
movie_dataset

Dataset({
    features: ['movie', 'dialogue', 'genre'],
    num_rows: 617
})

You can see that there are 617 distinct movies, and can continue to explore the data by looking at an example dialogue.

In [6]:
movie_dataset[3]["dialogue"][:150]

"Officers, there's your killer, do your duty, arrest him! ...so we kill someone famous and if we are caught, we are sent to mental hospital... I don't "

To move through the remainder of the notebook more quickly, select 200 samples.

In [7]:
# shuffle the data with fixed seed for reproducability
dataset = movie_dataset.shuffle(seed=42)

# select a sample of 200
dataset = dataset.select(range(200))

# save the dataset to disk
dataset.save_to_disk("movie_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

Delete all old variables that are no longer needed to free up memory with `del`.

In [8]:
del movie_dataset, movie_df

Make sure to release the memory after deleting the objects and variables that are no longer in use.

In [7]:
import gc
gc.collect()

<div class="alert alert-block alert-success">
<b>Conclusion</b>: In this section, you loaded a movie transcript dataset and converted it into a HuggingFace Dataset.
</div>

# <a name="2">2. Load and use a Large Language Model</a>
(<a href="#0">Go to top</a>)

[T5 (Text-To-Text Transfer Transformer)](https://github.com/google-research/text-to-text-transfer-transformer) is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, including machine translation, **document summarization**, question answering, and classification tasks (e.g., sentiment analysis). 

<div style="text-align: center;">
<img src="https://camo.githubusercontent.com/623b4dea0b653f2ad3f36c71ebfe749a677ac0a1/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f343030362f312a44304a31674e51663876727255704b657944387750412e706e67" width="700"/>
</div>

For more details have a look at the T5 documentation on HuggingFace 🤗 [here](https://huggingface.co/docs/transformers/model_doc/t5).

## 2.1. Loading T5

First, you have to download the T5 model using the `T5ForConditionalGeneration` class provided by the [HuggingFace 🤗 transformers library](https://github.com/huggingface/transformers) as well as the corresponding tokenizer `T5Tokenizer`. You can think of tokens as pieces of words that are required to pass information to LLMs. 

For English, **1 token is approximately 4 characters or 0.75 words**. This will be important to consider as LLMs are limited by the number of tokens they can pay attention to per prompt.

In [8]:
from transformers import T5ForConditionalGeneration

# load the model
model_t5 = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large",
    device_map={"": 0}, # this will load the model in GPU
    torch_dtype=torch.float32,
    return_dict=True
)

Together with a tokenizer the model can be used to generate text. So go ahead and initialize a tokenizer next.

In [9]:
from transformers import T5Tokenizer

# load the tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained(
    "google/flan-t5-large", 
    legacy=False, 
    max_length=512, 
    skip_special_tokens=True,
    return_tensors="pt",
    truncation=True
)

Let's create a prompt by joining an instruction to summarize text with the actual movie dialogue:

In [61]:
# create a prompt and use an example dialogue
inference_prompt = (
    "Summarize the following conversation from a movie script:  \n\n'''%s'''"
    % dataset[0]["dialogue"]
)

# let's look at the prompt but shorten the output to reduce the amount of text
print(inference_prompt[:235])

Summarize the following conversation from a movie script:  

'''I know.  Just be quick about it, will you? Do it right. Whistler, I -- No, we can treat the wounds -- Listen. You have to -- finish me off. You don't want me coming back. 


Have a look at what this looks like when converted to tokens:

In [62]:
print(tokenizer_t5(inference_prompt[:235]).input_ids)

[12198, 1635, 1737, 8, 826, 3634, 45, 3, 9, 1974, 4943, 10, 3, 31, 31, 31, 196, 214, 5, 1142, 36, 1704, 81, 34, 6, 56, 25, 58, 531, 34, 269, 5, 14883, 7, 14539, 6, 27, 1636, 465, 6, 62, 54, 2665, 8, 9699, 7, 1636, 12941, 5, 148, 43, 12, 1636, 1992, 140, 326, 5, 148, 278, 31, 17, 241, 140, 1107, 223, 5, 1]


The **number of tokens passed to an LLM through the tokenizer should not be greater than the number of tokens used in pre-training**. T5 was pre-trained using 512 input tokens, and with `truncation=True` all text beyond 512 tokens will be truncated.

## 2.2. Using T5 for inference on individual movie examples


Create an inference pipeline that encodes the input, generates a summary and then decodes the tokens to convert it back to text.

In [94]:
def generate_summary(prompt):
    # encode text
    encoded_tokens = tokenizer_t5(prompt, return_tensors="pt", skip_special_tokens=True)
    # generate summary
    generated_tokens = model_t5.generate(encoded_tokens.input_ids.to("cuda"), num_return_sequences=1, do_sample=False, num_beams=3, early_stopping=True, min_length=65, max_new_tokens=350, repetition_penalty=2.)
    # convert back
    output_text = tokenizer_t5.decode(generated_tokens.reshape(-1), skip_special_tokens=True)
    return output_text

Try the inference pipeline. The pipeline will return a list that you have to access to retrieve the LLM generated output.

In [95]:
from utils.model_utils import _format_llm_output

# pass the prompt to the pipeline and apply formatting
_format_llm_output(generate_summary(inference_prompt))

<div class="alert alert-block alert-info">"The Blood Tide" is based on the novel of the same name by Edgar Rice Burroughs. The film features an ensemble cast including Jason Lee, Karen Gillan, Deacon Frost, and Abraham Whistler. The movie opens with a narration by Edgar Rice Burroughs: "It's time for vampires to fight back."</div>

In [96]:
!nvidia-smi

Fri Nov 10 04:48:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   38C    P0    30W /  70W |  13677MiB / 15109MiB |     54%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

This summary looks okay but important characters that appear in the dialogue are not mentioned at all. This is due to the limited number of tokens T5 can 'keep track of'. Later, you will see a method that can help fix this issue.



<div class="alert alert-block alert-warning">
<b>Exercise 1</b>: Recreate the example above but for another movie.
</div>

In [49]:
##### complete your code here #####


###################################

Before proceeding, delete the prompts that were used for inference; e.g. <code>del inference_prompt</code> and also clear the GPU cache with <code>torch.cuda.empty_cache()</code>. 

In [50]:
del inference_prompt
gc.collect()
torch.cuda.empty_cache()

## 2.2. Using T5 for inference on all movie examples

The goal of this section is to summarize all movie dialogues. As previously mentioned, there is one very important caveat though - **Large Language Models are only able to pay attention to a limited number of tokens**. The amount of tokens an LLM can 'understand' is called 'context window'. Different LLMs will have different context windows. You can check out the context window size by trying to pass the full movie dialogue through the tokenizer and will see that you get a warning or you can inspect the model configurations. For more details have a look [here](https://huggingface.co/learn/nlp-course/chapter2/5?fw=tf#:~:text=With%20Transformer%20models%2C%20there%20is,asked%20to%20process%20longer%20sequences).

In [51]:
model_t5.config.__dict__["n_positions"]

512

As shown above, the context window for T5 models is 512 tokens. This means the movie transcript needs to be split into chunks of this lenght and summarised one by one. Then, a final summary needs to be created.

<div style="text-align: center;">
<img src="map_chain.png" width="900"/>
</div>

### 2.2.1. Chunking the movie transcripts
Let's start by creating chunks of the movie transcripts. One simple way to create chunks of text is to write a helper function and then apply this helper function to all the movies in the dataset.

In [52]:
def create_chunks(sample, CHUNK_LENGTH):
    """
    Splits a given text into chunks of a specified length and adds metadata to each chunk.
    """
    chunks = []
    # loop over entire text in steps of chunk size
    for c, i in enumerate(range(0, len(sample["dialogue"]), CHUNK_LENGTH)):
        # extract text
        chunk_text = sample["dialogue"][i : i + CHUNK_LENGTH]
        # create dictionary with the chunked text and metadata
        chunks.append(
            # remove uncompleted sentences with string split
            {"text": ".".join(chunk_text.split(".")[1:-1]).lstrip(), "metadata": {"page": c, "num_words": len(chunk_text)}}
        )
    # create new column
    sample["chunks"] = chunks
    return sample

Create the chunks for all the movie transcripts in the dataset with the help of `.map()`; this method efficiently applies the `create_chunks` function to all datapoints. Whenever you have additional parameters to pass to the model, you need to use a helper method, such as `partial`.

In [53]:
from functools import partial

# use partial to pass the arguments to the map function
dataset = dataset.map(partial(create_chunks, CHUNK_LENGTH=1650), batched=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

<div class="alert alert-block alert-warning">
<b>Exercise 2</b>: Think about how the chunking could be improved. Hint: Look for text splitters in the LangChain documentation.
</div>

In [54]:
###### write down ideas here ######


###################################

Now that the transcripts are chunked, let's start by setting up a prompt template for the intermediate (chunk) summaries. A prompt template is special construct that can parse input variables. Prompt templates can be applied to all the items in a dataset and help with consistency and reproducability. 

### 2.2.2. Prepare prompt templates and pipeline
Generally prompt templates can be very elaborate. In the case of T5, the prompts for pre-training all used the keyword 'summarize:', so this is what you should use.

In [55]:
from langchain import PromptTemplate

In [56]:
chunk_prompt_template = """You will be presented with a chunk of movie dialogue, summarize concisely. Summarize: {text}"""

chunk_prompt = PromptTemplate(
    template=chunk_prompt_template, input_variables=["text"]
)

You also need another prompt template to get the final summary.

In [57]:
combine_prompt_template = """The below are summaries from movie chunks; create a final summary. Summarize: {text}"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [65]:
!nvidia-smi

Fri Nov 10 04:25:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   27C    P0    24W /  70W |   8751MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

To get a summary from the model, use 🤗 HuggingFace [pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) or 🦜️🔗 LangChain's wrapper [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html). Pipelines are a great and easy way to use models for inference that offer a simple API dedicated to several tasks (e.g. [`summarization`](https://huggingface.co/transformers/v3.0.2/task_summary.html#summarization)).

In [71]:
from langchain.llms import HuggingFacePipeline

pipe = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-large",
    task="summarization",
    pipeline_kwargs={"max_new_tokens": 150,
                     "min_length":65,
                     "max_length":350,
                     "num_beams":5,
                     "early_stopping":True,
                     "do_sample":False,
                     # "num_return_sequences":1,
                     "repetition_penalty":2.,},
    model_kwargs={"return_dict":False,
                  "torch_dtype":torch.float32},
    device=0 # this will load the model in GPU
    )

In [72]:
!nvidia-smi

Fri Nov 10 04:26:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   27C    P0    30W /  70W |   9953MiB / 15109MiB |     45%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------



### 2.2.3. Create summaries of chunks and final summary

At this point now, you could apply the prompt template to all the chunks of movie transcripts to obtain your summaries, combine them back together and create a final summary. This would be a very lengthy and error-prone process, so instead make use of an increasingly popoular toolkit: [🦜️🔗 LangChain](https://python.langchain.com/docs/get_started/introduction).

🦜️🔗 LangChain has a [`Chain` module](https://python.langchain.com/docs/modules/chains/) which allows to create a sequence of calls to generic components (e.g. models or other chains). Luckily, text summarization is a very popular task, so there existis a predefined [summarization](https://python.langchain.com/docs/use_cases/summarization) method, called `load_summarize_chain`. This **chain will take the chunks, summarize them and then pass all the summaries to the LLM to create the final summary**.

In [73]:
from langchain.chains.summarize import load_summarize_chain

map_reduce_chain = load_summarize_chain(
    pipe,
    chain_type="map_reduce",
    map_prompt=chunk_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=False,
)


There is one more small caveat: LangChain expects all text to be passed as `Document` type following the 🦜️🔗 LangChain schema. So you will have to convert the chunks to the expected schema. Then you can test the summarization chain:

In [97]:
!nvidia-smi -q --display=MEMORY


==============NVSMI LOG==============

Timestamp                                 : Fri Nov 10 04:53:16 2023
Driver Version                            : 470.57.02
CUDA Version                              : 11.8

Attached GPUs                             : 1
GPU 00000000:00:1E.0
    FB Memory Usage
        Total                             : 15109 MiB
        Used                              : 13677 MiB
        Free                              : 1432 MiB
    BAR1 Memory Usage
        Total                             : 256 MiB
        Used                              : 5 MiB
        Free                              : 251 MiB



In [75]:
from langchain.schema import Document

sample_doc = [Document(page_content=split["text"], metadata=split["metadata"]) for split in dataset[0]["chunks"]]
    
# turn on verbosity for chain
map_reduce_chain.llm_chain.verbose = True

# run the summarization chain
map_reduce_example = map_reduce_chain({"input_documents": sample_doc})

# show the result
_format_llm_output(map_reduce_example["output_text"])



> Entering new LLMChain chain...
Prompt after formatting:
You will be presented with a chunk of movie dialogue, summarize concisely. Summarize: Just be quick about it, will you? Do it right. Whistler, I -- No, we can treat the wounds -- Listen. You have to -- finish me off. You don't want me coming back. Don't try to talk -- China Town. I need more serum.  What's all this? Going somewhere? Don't even start, old man. What took you so long? Wait. Get in. Youre leaving. It's not worth the risk. We can't trust her. Maybe not. I did some checking, she's a hematologist. Knowledge like that might come in handy. Stupidity. Just do it, old man. I had to increase the dose. You're building up a resistance to the serum -- She hasn't turned yet.  You can help her. You should've killed her, then. She's been bitten. Are we bringing home strays now? Whistler! It's because I'm human that I can do this. You're too human, Blade. You don't have a few minutes, Frost. You're wrong -- a few minutes more, a

<div class="alert alert-block alert-info">Frost is a half-breed who's been transformed into a human. Whistler wants to kill Frost, but Frost doesn't want him coming back. Frost and Whistler go to China Town to get more serum. Frost hasn't turned yet, so Whistler should've killed her. Frost and Whistler try to find a drink that's sweeter. Frost thinks the humans will never accept a half-breed like Blade.</div>

<div class="alert alert-block alert-warning">
<b>Exercise 3</b>: Recreate the example above but for another movie.
</div>

In [ ]:
##### complete your code here #####


###################################

The next step is to generate all summaries. Because the LLM has to generate summaries for every chunk of text, as well as a final summary, the time to create summaries for all movies in the dataset is approximately 6 hours. You can find the code below, but please skip this code cell and simply load the pre-generated summaries.

In [ ]:
# from utils.data_utils import _add_summaries

# # create summaries
# summaries_dataset = dataset.map(partial(_add_summaries, chain=map_reduce_chain), batched=False)

# # remove columns that are no longer needed
# summaries_dataset = summaries_dataset.remove_columns(["dialogue", "chunks"])

# # for backup save the dataset to local disk
# summaries_dataset.save_to_disk("summaries_dataset")

In [ ]:
!nvidia-smi

If you need to load in the dataset, you can do so with `load_from_disk('summaries_dataset')`. Make sure to import the method first with `from datasets import load_from_disk`.

In [78]:
from datasets import load_from_disk
summaries_dataset = load_from_disk('summaries_dataset_incl_toxic_rephrase')

<div class="alert alert-block alert-success">
<b>Conclusion</b>: At this point, you have summaries for all the movies and it is time to check whether those summaries contain any hate speech, slurs or toxic remarks.
</div>

In [79]:
del pipe, dataset
gc.collect()
torch.cuda.empty_cache()

In [80]:
!nvidia-smi

Fri Nov 10 04:43:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P0    26W /  70W |   7547MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 3. Evaluate LLM generated summaries for toxicity

To evaluate toxicity you can load the 🤗 [evaluate](https://huggingface.co/docs/evaluate/index) library and initialize a toxicity evaluator object. The model that will be used to evaluate toxicity is the [RoBERTa](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) model. RoBERTa was trained to detect toxicity on a dataset of approx. 40,000 entries, generated and labelled by trained annotators over four rounds
of dynamic data creation. Each hateful entry has fine-grained labels for the type and target of hate.

To evaluate the movie summary for toxicity, simply pass a list containing the summary text to the toxicity evaluator object. The aggregation parameter options are `None`, `maximum` and `ratio`.

In [81]:
from utils.eval_utils import _evaluate_toxicity

_evaluate_toxicity([summaries_dataset[0]["toxic_rephrase"]], aggregation_method=None)

[0.9922433495521545]

Have a look at another summary and calculate the score for that too. 

In [82]:
##### complete your code here #####


###################################

<div class="alert alert-block alert-warning">
<b>Exercise 4</b>: Calculate the max toxicity score for this example sentence: " ". Make sure to specify <code>aggregation_method="maximum"</code> as well.
</div>

In [83]:
##### complete your code here #####


###################################

Now that you evaluated for a few movies manually, it is time to evaluate all movie summaries and obtain a list of toxicity scores.

In [84]:
from utils.eval_utils import _add_toxicty_column
   
summaries_dataset = _add_toxicty_column(summaries_dataset)

<div class="alert alert-block alert-warning">
<b>Exercise 5</b>: Try to calculate mean toxicity for two different movie genres.
</div>

In [85]:
##### complete your code here #####


###################################

In [86]:
!nvidia-smi

Fri Nov 10 04:43:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   26C    P0    24W /  70W |   7547MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

<div class="alert alert-block alert-success">
<b>Conclusion</b>: We have seen that some summaries are toxic and would like to remediate this. In general, to update the output that is generated by LLMs, a technique called 'fine-tuning' is used. Fine-tuning requires a set of examples and the corresponding ground truth. In theory, it would be possible to ask human evaluators to look at multiple different versions of movie dialogue summaries and then rank them. However, this is time consuming and therefore it makes sense to repurpose the toxicity model and use the toxicity values as signal for what is considered good (no toxicity) and bad (toxicity). This helper model, is the so-called reward model.
</div>

# 4. Reduce toxicity using a Direct Optimization Policy (DPO)

To include human feedback, the first step is to ensure the data is in-distribution for the DPO algorithm. Supervised fine-tuning (or SFT for short) can help with this.  The following code-snippet takes care of all the data pre-processing and training for you; have a look at the documentation [here](https://huggingface.co/docs/trl/sft_trainer) and more details about the SFTTrainer class [here](https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py). For a full overview of the method, have a look [here](https://huggingface.co/blog/dpo-trl).

In [87]:
ds = summaries_dataset.train_test_split(train_size=1, test_size=50, seed=0)

In [88]:
from transformers import TrainingArguments
from trl import SFTTrainer

EPOCHS = 2
LEARNING_RATE = 2e-4

sft_training_args = TrainingArguments(
    output_dir="sfft-model",
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="paged_adamw_8bit",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    logging_strategy="epoch",  # this will print loss at every epoch
)

# instantiate the trainer
trainer = SFTTrainer(
    model=model_t5,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    dataset_text_field="summary",
    max_seq_length=512,
    tokenizer=tokenizer_t5,
    dataset_batch_size=4,
    args=sft_training_args,  # HF Trainer arguments
)

model_t5.config.use_cache = False

# train the model to recognize the data domain for movies
trainer.train()

# specify where to save the pre-trained (domain adapted) SFT-model
trainer.model.save_pretrained("sft-domain-pretrained")

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

{'loss': 0.0269, 'learning_rate': 0.0001, 'epoch': 1.0}
{'loss': 0.0079, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 1.9151, 'train_samples_per_second': 1.044, 'train_steps_per_second': 1.044, 'train_loss': 0.017404975835233927, 'epoch': 2.0}


You have trained the model on the movie summaries and it is time to prepare for the preference adaptation. For this, the model needs extra layers of trainable parameters and also some post-processing to help with memory usage and stability.

The DPO trainer expects a model of `AutoModelForCausalLM`, compared to PPO that expects `AutoModelForCausalLMWithValueHead` for the value function.

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# configure the layers for LoRa
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
 
# add adaptable layers to the SFT-model
base_model = get_peft_model(trainer.model, peft_config)

# specify where to save the pre-trained (domain adapted) model
base_model.save_pretrained("adapters", save_peft_format=True)

from peft import PeftModelForCausalLM
from trl import create_reference_model

m = T5ForConditionalGeneration.from_pretrained(
    "sft-domain-pretrained",  # location of saved SFT model
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32,
    device_map={"": 0},
)

model = PeftModelForCausalLM.from_pretrained(m, "adapters", is_trainable=True)
model_ref = create_reference_model(model)


The DPO model will be trained to directly optimize the preference of which sentence is the most relevant, given two sentences. The DPO trainer expects a very specific format for the dataset. The entries should be named:

- `prompt`
- `chosen`
- `rejected`


In [ ]:
from typing import Dict
from functools import partial

def return_prompt_and_responses(samples, batch_multiplier) -> Dict[str, str]:
    """
    Create correct format for DPO steps.
    """
    return {
        "prompt": ["""Write a summary of this chunk of movie dialogue delimited by triple backquotes that includes the main points and any important details."""]*batch_multiplier,
        "chosen": samples["summary"],   # rated better than k
        "rejected": samples["toxic_rephrase"], # rated worse than j
            }

original_columns = ds["train"].column_names


BATCH_DATA = 4

# reshape the dataset to format DPO expects
dpo_ds = ds["train"].map(partial(return_prompt_and_responses, batch_multiplier=BATCH_DATA),
                        batched=True,
                        batch_size=BATCH_DATA,
                        remove_columns=original_columns)


Once we have the dataset sorted the DPO loss is essentially a supervised loss which obtains an implicit reward via a reference model and thus at a high-level the DPOTrainer requires the base model we wish to optimize as well as a reference model:

In [ ]:
from trl import DPOTrainer

EPOCHS = 4
LEARNING_RATE = 2e-4

dpo_training_args = TrainingArguments(
    output_dir="feedback-model-new",
    remove_unused_columns=False,
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="paged_adamw_8bit",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    logging_strategy="epoch",  # this will print loss at every epoch
)

dpo_trainer = DPOTrainer(
    model,  # base model from SFT pipeline
    model_ref,  # a copy of the SFT trained base model
    beta=0.1,  # temperature hyperparameter of DPO
    train_dataset=dpo_ds,  # dataset prepared above
    tokenizer=tokenizer_t5,  # tokenizer
    args=dpo_training_args,  # training arguments e.g. batch size, lr, etc.
    max_length=150,
    max_prompt_length=300,
    max_target_length=128,
)

In [ ]:
dpo_trainer.train()

In [ ]:
# enable inference
dpo_trainer.model.config.use_cache = True

In [ ]:
encoded = tokenizer_t5(summaries_dataset[0]["toxic_rephrase"], return_tensors="pt")

In [ ]:
summaries_dataset[0]

In [ ]:
dpo_output = dpo_trainer.model.generate(
    input_ids=encoded["input_ids"].to("cuda"),
    max_new_tokens=150,
    do_sample=True,
    top_p=0.8)

In [ ]:
tokenizer_t5.decode(dpo_output[0].detach().cpu().numpy(),
                    skip_special_tokens=False,
                    clean_up_tokenization_spaces=False)

<div class="alert alert-block alert-warning">
<b>Exercise 6</b>: Compare summaries from the DPO model to the reference model.
</div>

In [ ]:
ref_output = model_ref.generate(
    input_ids=encoded["input_ids"].to("cuda"),
    max_new_tokens=450,
    do_sample=True,
    top_p=0.6)

tokenizer_t5.decode(ref_output[0].detach().cpu().numpy(),
                    skip_special_tokens=False,
                    clean_up_tokenization_spaces=False)

# 5. Evaluate and refine the model

In [ ]:
# we have seen anecdotal, let's check if overall it actually is better
# average score across all data points with reference model vs DPO model?
# try doing this on genre to find larger differences


# Next steps

In [ ]:
from profanity_check import predict

@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        prediction = predict([value])
        if prediction[0] == 1:
            return FailResult(
                error_message=f"The result contains profanity and will be filtered.",
                fix_value="",
            )
        return PassResult()

In [ ]:
rail_spec_metric = """
<rail version="0.1">
    <output>
        <string description="Summarization" format="is-profanity-free" name="translated_statement" on-fail-is-profanity-free="filter"></string>
    </output>

    <prompt>
    Translate the given statement into English language:
    ${statement_to_be_translated}
    ${gr.complete_json_suffix}
    \n\nAssistant:
    </prompt>

</rail>
"""

In [ ]:
guard_metric = gd.Guard.from_rail_string(rail_spec_metric)

raw_llm_response, validated_response = guard_metric(
    llm_api=bedrock_llm,
    prompt_params={"statement_to_be_translated": "Ich hasse dich."},
)

print(f"Validated Output: {validated_response}")

In [ ]:
print(guard_metric.state.most_recent_call.tree)

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.serializers import JSONSerializer


model_id, model_version, = (
    "huggingface-text2text-flan-t5-xxl",
    "*",
)


inference_instance_type = "ml.g5.2xlarge"
my_model = JumpStartModel(model_id=model_id)
# deploy the model to 1 single instance of type inference_instance_type


# this takes 10-15 minutes to deploy
predictor = my_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type
)

# example prompt
prompt = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 50,
        "return_full_text": True,
        "do_sample": True,
        "top_k": 10,
        "stop": ["<|endoftext|>", "</s>"],
    },
}

response = predictor.predict(payload)
print(response[0]["generated_text"])
